In [1]:
import torch
#import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms

!pip install torchsummary
from torchsummary import summary

from ezio.data import dataset 
from ezio.runner import engine
from ezio.utils import loss

In [2]:
## Create train and test transforms
train_transforms = transforms.Compose(
    [transforms.RandomAffine(7, (0.1, 0.1), (0.9, 1.1), 7),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))])

test_transforms = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784))])

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
train_set, valid_set = dataset.cifar10_dataset(train_transforms, test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader, valid_loader = dataset.create_loaders(train_set, valid_set, train_batch_size=128, valid_batch_size=32, num_workers=4)

In [6]:
from ezio.model.QuizDNN import QuizDNN
model = QuizDNN().to(device)

In [7]:
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
              ReLU-2           [-1, 64, 32, 32]               0
       BatchNorm2d-3           [-1, 64, 32, 32]             128
            Conv2d-4           [-1, 64, 32, 32]          36,864
              ReLU-5           [-1, 64, 32, 32]               0
       BatchNorm2d-6           [-1, 64, 32, 32]             128
           Dropout-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
              ReLU-9           [-1, 64, 32, 32]               0
      BatchNorm2d-10           [-1, 64, 32, 32]             128
          Dropout-11           [-1, 64, 32, 32]               0
        MaxPool2d-12           [-1, 64, 16, 16]               0
           Conv2d-13           [-1, 64, 16, 16]          36,864
             ReLU-14           [-1, 64,

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=0, verbose=True, factor=0.5)
loss_function = loss.cross_entropy_loss()

In [ ]:
train_acc, train_losses, test_acc, test_losses = engine.fit(model, train_loader, valid_loader, loss_function, device, optimizer, 40, scheduler)

  0%|                                                                                          | 0/391 [00:00<?, ?it/s]

EPOCH: 1


Loss=1.6280043125152588 Batch_id=76 Accuracy=31.00:  20%|█████▋                       | 77/391 [00:40<02:07,  2.47it/s]